<p align="left">
  <img src="./logo_UTN.svg" />
</p>


# **Teoría de Circuitos II - R4001 - 2023**
## Trabajo Semanal 4
### Autor: Bruno Glecer

# Consigna

La tarea semanal consiste en diseñar un filtro pasa-altos que respete la siguiente plantilla.

<div>
    <img src="plantilla.png" width="400"/>
</div>

Donde los parametros son los siguientes:
- $\large\alpha_{max} = 1\mathrm{dB}$
- $\large\alpha_{min} = 30\mathrm{dB}$
- $\large f_{p} = 40\mathrm{kHz}$
- $\large f_{s} = 10\mathrm{kHz}$

Se piden las siguientes consignas:

1. Obtener la transferencia de máxima planicidad del filtro requerido.
2. Obtener el diagrama de polos y ceros, y un bosquejo de la respuesta en frecuencia. Compare el diagrama de polos y ceros con el del filtro pasabajo prototipo.
3. Implementar el circuito normalizado con estructuras pasivas. (Puede utilizar dispositivos activos para separar secciones).
4. Reemplace los inductores en las estructuras pasivas mediante el GIC de Antoniou, en la configuración que considere más apropiada.



# 1. Transferencia

Comenzamos normalizando el filtro usando $\omega_p$ como norma y utilizamos el nucleo $p = K(s) = \frac{1}{s}$

In [1]:
import numpy as np

#Normalizacion de valores

wp = 40*10**3
ws = 10*10**3
alpha_max = 1
alpha_min = 30

norma_w = wp

wp_n = wp/norma_w
ws_n = ws/norma_w


#Aplicacion del nucleo de transformacion

wp_lp = 1.0/wp_n
ws_lp = 1.0/ws_n

print(f"wp_lp = {wp_lp}    ws_lp = {ws_lp}")

wp_lp = 1.0    ws_lp = 4.0


Continuamos encontrando la función de transferencia del filtro prototipo low-pass.

Al ser un filtro de máxima planicidad, el filtro low pass tendra una función de transferencia de potencia:

$|T_{LP}(\Omega)|^2 = \dfrac{1}{1 + \varepsilon^2 \Omega^{2n}}$

In [2]:
#Calculo de epsilon

epsilon = np.sqrt(10**(alpha_max/10) - 1)

#Calculo de orden

n = 1

while (10*np.log10(1 + epsilon**2 * ws_lp ** (2*n))) < alpha_min:
    n += 1

print(f"epsilon = {epsilon}    n = {n}")

epsilon = 0.5088471399095875    n = 3


Seguimos buscando las raices de la funcion de transferencia de potencia de este prototipo y separando los polos que esten en el semiplano compljeo izquierdo ($\Sigma < 0$)

In [3]:
den_tlp2 = np.zeros(2*n)
den_tlp2[0] = 1
den_tlp2[2*n - 1] = epsilon**2

polos_tlp2 = np.roots(den_tlp2)
polos_tlp = polos_tlp2[np.real(polos_tlp2) < 0]

print(f"Polos de transferencia: {polos_tlp}")

Polos de transferencia: [-0.76319398+0.j         -0.23583991+0.72584061j -0.23583991-0.72584061j]


Utilizando estos polos, que consisten de dos polos conjugados y un polo en el eje real, puedo construir mi funcion de transferencia del prototipo para que cumpla $|T_{LP}(p)|^2 = T_{LP}(p) \cdot T_{LP}(-p)$ 

In [4]:
from scipy import signal
from pytc2.sistemas_lineales import analyze_sys, pretty_print_bicuad_omegayq,tfcascade,pretty_print_lti
from sympy.abc import p, s, T
import sympy
from IPython.display import display
#Calculo transferencia del prototipo

t_lp_num = [1]
t_lp_den = np.poly(polos_tlp)
t_lp_den /= t_lp_den[0]
t_lp_num /= t_lp_den[0]

print("Transferencia T_LP(p): ")

pretty_print_lti(t_lp_num,t_lp_den)

#Calculo trasnferencia del objetivo

t_lp = sympy.Eq(T, sympy.Poly(t_lp_num, p)/sympy.Poly(t_lp_den, p))

nucleo_p_s = sympy.Eq(p, 1/s) #Nucleo de transformacion

t_hp = sympy.solve([t_lp, nucleo_p_s], [T, p], dict=True)[0][T]

t_hp_num = sympy.fraction(t_hp, s)[0]
t_hp_den = sympy.fraction(t_hp, s)[1]

t_hp_num = t_hp_num.as_poly(domain='C').as_expr()
t_hp_den = t_hp_den.as_poly(domain='C').as_expr()

t_hp_num /= t_hp_den.coeff(s,n)
t_hp_den /= t_hp_den.coeff(s,n)

t_hp = t_hp_num/t_hp_den

t_hp

Transferencia T_LP(p): 


<IPython.core.display.Math object>

2.2495476385401*s**3/(1.0*s**3 + 2.12008222621325*s**2 + 2.77790743405706*s + 2.2495476385401)

In [5]:
signal.butter(n, 1, btype='highpass', analog=True)

#[x for (x,i) in zip(signal.butter(n, 1, btype='highpass', analog=True)[1], [0,1,2,3])]

(array([1., 0., 0., 0.]), array([1., 2., 2., 1.]))